In [1]:
from Classification import classification_class as classification
from Features_extraction import feature_extraction_class as feature_extraction
from Data_processing.images_processing_class import ImagesProcessing
from utils import perform_classification, best_preprocessing

import pandas as pd
import numpy as np
import cv2
import json

In [2]:
metrics_results_best_methods = {
        "accuracy" : {},
        "f1-score" : {},
        "recall" : {},
        "precision" : {},
        "roc_auc" : {}
        }

name_best_models = {}

In [3]:
data_normal_path = "Data/normal/"
data_pothole_path = "Data/potholes/"

image_process = ImagesProcessing(folder_normal=data_normal_path, folder_potholes=data_pothole_path, img_size=(256, 256))
images = image_process.images
Y = image_process.labels
df_Y = pd.DataFrame(Y)

# I. Unprocessed images

In [4]:
env_feature_extraction_unprocessed_images = feature_extraction.FeatureExtraction(images)

In [ ]:
methods = {
    "SIFT + BoW": env_feature_extraction_unprocessed_images.method_SIFT,
    "ORB + BoW": env_feature_extraction_unprocessed_images.method_ORB,
    "EDGE": env_feature_extraction_unprocessed_images.method_EDGE
}

for method_name, method_function in methods.items():
    metrics_results_best_methods = perform_classification(
        method_function, method_name, df_Y, name_best_models, metrics_results_best_methods
    )


# II. Find best configurations for preprocessing

In [5]:
# TODO: Change this filter with the good association method-filter find in articles

filters = {
    "SIFT": "gaussian",
    "ORB": "median",
    "EDGE": "bilateral"
}

best_configs = {}

for method_name, filter_name in filters.items():
    best_config, all_results = best_preprocessing(
        image_process, getattr(env_feature_extraction_unprocessed_images, f"method_{method_name}"), filter_name, method_name, n_iter=30
    )
    best_configs[method_name] = best_config

Extracting SIFT Features
Best preprocessing configuration for SIFT: {'filter': 'gaussian', 'filter_params': ((3, 3), 0), 'histogram': 'standard', 'gamma': 0.8, 'normalize': False, 'f1-score': 0.7926342950973493}
Extracting ORB Features
Best preprocessing configuration for ORB: {'filter': 'median', 'filter_params': (3,), 'histogram': 'standard', 'gamma': 0.8, 'normalize': False, 'f1-score': 0.5966649879693358}
Extracting Edge features
Best preprocessing configuration for EDGE: {'filter': 'bilateral', 'filter_params': (5, 50, 50), 'histogram': 'standard', 'gamma': 0.8, 'normalize': False, 'f1-score': 0.8320987654320987}


In [ ]:
with open("best_configs_processing.json", "w") as f:
    json.dump(best_configs, f, indent=4)

# III. Application of the best preprocessing

In [ ]:
with open("best_configs_methods.json", "r") as f:
    best_configs_methods = json.load(f)

In [ ]:
for method_name, best_config in best_configs.items():
    best_params_method = best_configs_methods[method_name]

    print("=====================================================================")
    print(f"\033[1mComputing results for {method_name}\033[0;0m")
    print(f"Applying best preprocessing :\n {best_config} \n")
    print(f"Applying best method parameters:\n {best_params_method} \n")

    image_process.apply_preprocessing(best_config)

    images=image_process.images
    
    env_feature_extraction = feature_extraction.FeatureExtraction(images)
    feature_extraction_method = getattr(env_feature_extraction, f"method_{method_name}")
    
    method_name_preprocessed = f"{method_name} + {filters[method_name]}"

    metrics_results_best_methods = perform_classification(
        feature_extraction_method, method_name_preprocessed, best_params_method, df_Y, name_best_models, metrics_results_best_methods
    )


Applying best preprocessing for SIFT: {'filter': 'gaussian', 'filter_params': ((5, 5), 1), 'histogram': 'none', 'gamma': 'none', 'normalize': False, 'f1-score': 0.7836818851251841}
Extracting SIFT Features
Performing Classification for SIFT + gaussian
Best method name for SIFT + gaussian: RandomForest
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.81      0.80        86
           1       0.65      0.60      0.62        50

    accuracy                           0.74       136
   macro avg       0.71      0.71      0.71       136
weighted avg       0.73      0.74      0.73       136

Applying best preprocessing for ORB: {'filter': 'median', 'filter_params': (5,), 'histogram': 'clahe', 'gamma': 1.0, 'normalize': False, 'f1-score': 0.6053571428571429}
Extracting ORB Features
Performing Classification for ORB + median
Best method name for ORB + median: LogReg
Classification Report:
              precision    recall  f1-score   

# IV. Overall results

In [10]:
df_best_models_all_datasets_macro_f1 = pd.DataFrame(metrics_results_best_methods)
df_best_models_all_datasets_macro_f1.insert(0, 'Best models', name_best_models)
df_best_models_all_datasets_macro_f1.style.highlight_max(axis=0, subset=df_best_models_all_datasets_macro_f1.columns[1:])


,Best models,accuracy,f1-score,recall,precision,roc_auc
SIFT + BoW,LogReg,0.801471,0.783682,0.780233,0.788071,0.780233
ORB + BoW,LogReg,0.617647,0.605357,0.613953,0.606800,0.613953
EDGE,LogReg,0.794118,0.787500,0.803721,0.784656,0.803721
SIFT,RandomForest,0.750000,0.726326,0.722791,0.731401,0.722791
ORB,LogReg,0.558824,0.552632,0.567442,0.562771,0.567442
SIFT + gaussian,RandomForest,0.735294,0.710227,0.706977,0.714976,0.706977
ORB + median,LogReg,0.558824,0.552632,0.567442,0.562771,0.567442
EDGE + bilateral,LogReg,0.794118,0.787500,0.803721,0.784656,0.803721
